In [1]:
import tensorflow as tf
import numpy as np
import cv2
import scipy.io as sio

In [2]:
# 文件的读取
f = open('Yale_64x64.mat','rb')
mdict = sio.loadmat(f)
# fea gnd 是由数据决定的
train_data = mdict['fea']
train_label = mdict['gnd']

# test
print(train_data.shape)
print(train_label.shape)
# print(train_data)
# print(train_label)
# cv2.imshow("img",train_data[0,:].reshape(64,64))
# cv2.waitKey(0)
np.random.seed(5)
train_data = np.random.permutation(train_data)
np.random.seed(5)
train_label = np.random.permutation(train_label)
test_data = train_data[:64]
test_label = train_label[:64]

np.random.seed(100)
test_data = np.random.permutation(test_data)
np.random.seed(100)
test_label = np.random.permutation(test_label)

# train_data
train_data = np.float32(train_data.reshape((train_data.shape[0],64,64,1)))/255

# train_label
train_label_new = np.zeros((165,15))
for i in range(165):
    j = int(train_label[i,0])-1
    train_label_new[i,j] = 1
    
test_data = np.float32(test_data.reshape((test_data.shape[0],64,64,1)))/255
test_label_new = np.zeros((64,15))
for i in range(64):
    j = int(test_label[i,0])-1
    test_label_new[i,j] = 1


(165, 4096)
(165, 1)


In [4]:
data_input = tf.placeholder(tf.float32,[None,64,64,1])
label_input = tf.placeholder(tf.float32,[None,15])

# cnn tf.layer.conv2d tf.nn.conv2d
w0 = tf.Variable(tf.random_normal([2,2,1,32],stddev=0.1,dtype=tf.float32))
b0 = tf.Variable(tf.constant(0.1,shape=[32]))
layer1 = tf.nn.relu(tf.nn.conv2d(data_input,w0,strides=[1,1,1,1],padding='SAME'))
# layer1 = tf.layers.conv2d(inputs=data_input,filters=32,kernel_size=2,strides=1,padding='SAME',activation=tf.nn.relu)

layer1_pool = tf.nn.max_pool(layer1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
layer2 = tf.reshape(layer1_pool,[-1,32*32*32])

w2 = tf.Variable(tf.random_normal([32*32*32,1024],stddev=0.1,dtype=tf.float32))
b2 = tf.Variable(tf.constant(0.0,shape=[1024],dtype=tf.float32))
fc = tf.nn.relu(tf.matmul(layer2,w2)+b2)

w3 = tf.Variable(tf.random_normal([1024,15],stddev=0.1,dtype=tf.float32))
b3 = tf.Variable(tf.constant(0.0,shape=[15],dtype=tf.float32))
output = tf.matmul(fc,w3)+b3

loss = tf.losses.softmax_cross_entropy(onehot_labels=label_input,logits=output)
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
# 概率
accuracy = tf.metrics.accuracy(labels=tf.argmax(label_input,axis=1),predictions=tf.argmax(output,axis=1))[1]

In [5]:
with tf.Session() as sess:
    sess.run(tf.group(tf.global_variables_initializer(),tf.local_variables_initializer()))
#     print(sess.run(output,feed_dict={data_input:train_data}).shape)

    for i in range(100):
        sess.run(train,feed_dict={data_input:train_data,label_input:train_label_new})
        acc = sess.run(accuracy,feed_dict={data_input:test_data,label_input:test_label_new})
        print('acc:%.2f',acc)
#         print(sess.run(output,feed_dict={data_input:test_data}))
#         print(test_label_new)

acc:%.2f 0.046875
acc:%.2f 0.078125
acc:%.2f 0.114583336
acc:%.2f 0.12890625
acc:%.2f 0.140625
acc:%.2f 0.1484375
acc:%.2f 0.15625
acc:%.2f 0.1640625
acc:%.2f 0.171875
acc:%.2f 0.18125
acc:%.2f 0.18892045
acc:%.2f 0.1953125
acc:%.2f 0.20192307
acc:%.2f 0.20870535
acc:%.2f 0.21458334
acc:%.2f 0.21972656
acc:%.2f 0.22610295
acc:%.2f 0.23350695
acc:%.2f 0.24095395
acc:%.2f 0.2484375
acc:%.2f 0.2559524
acc:%.2f 0.26491478
acc:%.2f 0.2737772
acc:%.2f 0.28190103
acc:%.2f 0.290625
acc:%.2f 0.2986779
acc:%.2f 0.30844906
acc:%.2f 0.31863838
acc:%.2f 0.3292026
acc:%.2f 0.34010416
acc:%.2f 0.3518145
acc:%.2f 0.36328125
acc:%.2f 0.37405303
acc:%.2f 0.3841912
acc:%.2f 0.39375
acc:%.2f 0.4027778
acc:%.2f 0.41300675
acc:%.2f 0.421875
acc:%.2f 0.43108973
acc:%.2f 0.4390625
acc:%.2f 0.44817072
acc:%.2f 0.45572916
acc:%.2f 0.46402615
acc:%.2f 0.47123578
acc:%.2f 0.47881943
acc:%.2f 0.4857337
acc:%.2f 0.49235374
acc:%.2f 0.4983724
acc:%.2f 0.50510204
acc:%.2f 0.510625
acc:%.2f 0.5168505
acc:%.2f 0.521935